# Animations of 3D rotation and different network topologies using plotly

In [76]:
import networkx as nx
import numpy as np
import random
import plotly.io as pio
pio.renderers.default = "browser"
import plotly.graph_objects as go

In [77]:
# Parameters dictionary
params = {
    "num_nodes": 100,
    "initial_infected": 3,
    "spread_chance": 0.1,
    "recovery_chance": 0.02,
    "network_type": "erdos_renyi",  # Added "powerlaw_cluster" as an option
    "erdos_renyi_p": 0.15,          # Probability for Erdős-Rényi
    "barabasi_albert_m": 2,        # Number of edges to attach for Barabási-Albert
    "watts_strogatz_k": 4,         # Each node is connected to k neighbors in Watts-Strogatz
    "watts_strogatz_p": 0.1,       # Rewiring probability for Watts-Strogatz
    "powerlaw_cluster_m": 2,       # Number of edges to attach for Powerlaw-Cluster
    "powerlaw_cluster_p": 0.3      # Probability of adding a triangle for Powerlaw-Cluster
}

# Create the graph based on the selected topology
if params["network_type"] == "erdos_renyi":
    G = nx.erdos_renyi_graph(params["num_nodes"], params["erdos_renyi_p"])
elif params["network_type"] == "barabasi_albert":
    G = nx.barabasi_albert_graph(params["num_nodes"], params["barabasi_albert_m"])
elif params["network_type"] == "watts_strogatz":
    G = nx.watts_strogatz_graph(params["num_nodes"], params["watts_strogatz_k"], params["watts_strogatz_p"])
elif params["network_type"] == "powerlaw_cluster":
    G = nx.powerlaw_cluster_graph(params["num_nodes"], params["powerlaw_cluster_m"], params["powerlaw_cluster_p"])
else:
    raise ValueError("Invalid network type specified in params.")

In [78]:
# Generate 3D positions for nodes
pos = nx.spring_layout(G, dim=3)
nodes = np.array([pos[node] for node in G.nodes])

# Initialize node states
states = {node: 'S' for node in G.nodes}  # 'S' = Susceptible, 'I' = Infected, 'R' = Recovered
for node in random.sample(list(G.nodes), params["initial_infected"]):
    states[node] = 'I'

In [79]:
# Update function for the simulation
def update_states():
    global states
    new_states = states.copy()
    for node in G.nodes:
        if states[node] == 'I':  # Infected
            # Try to infect neighbors
            for neighbor in G.neighbors(node):
                if states[neighbor] == 'S' and random.random() < params["spread_chance"]:
                    new_states[neighbor] = 'I'
            # Try to recover
            if random.random() < params["recovery_chance"]:
                new_states[node] = 'R'
    states.update(new_states)

# Create a single frame of the 3D plot
def create_3d_frame():
    # Create edge traces
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges:
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='gray', width=2),
        hoverinfo='none'
    )

    # Create node traces
    node_x = []
    node_y = []
    node_z = []
    node_color = []
    for node in G.nodes:
        x, y, z = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        color = "blue" if states[node] == 'S' else "red" if states[node] == 'I' else "green"
        node_color.append(color)

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(size=8, color=node_color, opacity=0.8),
        hoverinfo='text'
    )

    return [edge_trace, node_trace]

# Generate frames for animation
frames = []

# Add the initial state as the first frame
initial_frame_data = create_3d_frame()
frames.append(go.Frame(data=initial_frame_data))

while True:  # Continue until the stopping condition is met
    update_states()
    
    # Check if there are no infected nodes ('I')
    if all(state != 'I' for state in states.values()):
        break  # Stop generating frames if no nodes are infected
    
    # Create a frame for the current state
    frame_data = create_3d_frame()
    frames.append(go.Frame(data=frame_data))

# Create the figure
fig = go.Figure(
    data=initial_frame_data,  # Set the initial state as the starting data
    frames=frames,  # Add all frames to the animation
    layout=go.Layout(
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)]),
                dict(label="Pause",
                     method="animate",
                     args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")])
            ]
        )]
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(showbackground=False),
        yaxis=dict(showbackground=False),
        zaxis=dict(showbackground=False),
    ),
    margin=dict(l=0, r=0, t=0, b=0),
)

# Show the animation
fig.show()

In [80]:
# Save as an interactive HTML file
fig.write_html("erdos_renyi/erdos_renyi_plot.html")